In [22]:
##Step1 ranking
import os
import subprocess
file_name="MyFile8"
ranking_method="randomForests"
cmd=["java","-jar","RankLib-2.13.jar","-rank","uploadedFiles/"+file_name+".txt","-load","trainedModels/"+ranking_method+".txt","-norm","zscore","-indri","myNewRankedLists.txt"]
output=[]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
for line in proc.stdout.readlines():
    x=str(line)
    x=x[2:len(x)-3]
    x=x.replace('\\t',' ')
    print(x)
    output.append(x)
score_data={}
label_data = {}
query={}
query_name = {}
file1 = open("uploadedFiles/"+file_name+".txt","r")
for x in file1:
    curr_row=x.split(" ")
    qid=int(curr_row[1].split(":")[1])
    date=float(curr_row[14].split(":")[1])
    index2=int(curr_row[26].split("_")[0])-1
    if qid not in score_data:
        label_data[qid] = {}
        score_data[qid] = []
        query[qid]=curr_row[26].split("_")[2]
        rem = x.split("=")[1]
        i = rem.find("of")
        rem = rem[i+3:]
        rem = rem[:len(rem)-1]
        query_name[qid] = rem
    label_data[qid][index2+1] = int(curr_row[0])
    score_data[qid].append([0,date,index2+1])
file1.close()
f = open("myNewRankedLists.txt","r")
f1 = f.readlines()
ans = {}
for x in f1:
    y = x.split()
    value = y[len(y)-2]
    q_id = y[0]
    z = x.split('_')
    i = z[0]
    i = i.split()
    # print(i)
    i = i[4]
    for v in range(len(score_data[int(q_id)])):
        # print(score_data[int(q_id)][v][2])
        # print(i)
        if score_data[int(q_id)][v][2] == int(i):
            score_data[int(q_id)][v][0] = float(value)
            break
for id in score_data.keys():
    score_data[id].sort(key = lambda sub: (-sub[0], sub[1]))
inputInfo=output[6]
inputInfo=inputInfo[1:len(inputInfo)-1]
modelInfo=output[4]
file1 = open("myNewRankedLists.txt","w")
for qid in score_data.keys():
    cnt = 1
    for doc in score_data[qid]:
        s=str(qid)+" Q0 docid = "+str(doc[2])+"_of_"+query_name[qid]+" "+str(cnt)+" "+str(doc[0])+" indri\n"
        cnt = cnt+1
        file1.write(s)
file1.close()

\r
Discard orig. features\r
Model file: trainedModels/randomForests.txt\r
Feature normalization: zscore\r
Model:  Random Forests\r
\rReading feature file [uploadedFiles/MyFile8.txt]: 0... \rReading feature file [uploadedFiles/MyFile8.txt]: 0... \rReading feature file [uploadedFiles/MyFile8.txt]... [Done.]            \r
(11 ranked lists, 12723 entries read)\r


In [19]:
s = " 3_of_4_Years_of_FAN\n"
i = s.find("of")
s = s[i+3:]
s = s[:len(s)-1]
print(s)

4_Years_of_FAN


In [3]:
#Step2 ranking(top-n)
#get top-n tweets id
n=150
file = open("myNewRankedLists.txt","r")
lines = file.readlines()
counter=0
top_n={}
for line in lines:
    line_data=line.split(" ")
    print(line_data[4])
    qid=line_data[0]
    if qid not in top_n:
        top_n[qid]=[]
        counter=0
    if counter<n:
        top_n[qid].append(line_data[4])
    counter=counter+1

IndexError: list index out of range

In [31]:
#get details of top-n tweets and make new file for testing into second phase
file1 = open("uploadedFiles/"+file_name+".txt","r")
tweets={}
for x in file1:
    curr_row=x.split(" ")
    l=len(curr_row)
    doc_id=curr_row[l-1]
    doc_id=doc_id[:len(doc_id)-1]
    tweets[doc_id]=x
    
file1 = open("uploadedFiles/intermidiate_"+file_name+".txt","w")
for key in top_n.keys():
    for doc in top_n[key]:
        file1.write(tweets[doc])
file1.close()

In [33]:
#second phase ranking 
##Step1 ranking
import os
import subprocess
intermidiate_file_name="intermidiate_"+file_name
intermidiate_ranking_method="intermidiate_"+ranking_method
cmd=["java","-jar","RankLib-2.13.jar","-rank","uploadedFiles/"+intermidiate_file_name+".txt","-load","trainedModels/"+intermidiate_ranking_method+".txt","-norm","zscore","-indri","intermidiateRankedLists.txt"]
output=[]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
for line in proc.stdout.readlines():
    x=str(line)
    x=x[2:len(x)-3]
    x=x.replace('\\t',' ')
    print(x)
    output.append(x)


Discard orig. features
Model file: trainedModels/intermidiate_randomForests.txt
Feature normalization: zscore
Model:  Random Forests
\rReading feature file [uploadedFiles/intermidiate_MyFile8.txt]: 0... \rReading feature file [uploadedFiles/intermidiate_MyFile8.txt]... [Done.]            
(11 ranked lists, 1650 entries read)


In [40]:
file = open("myNewRankedLists.txt","r")
lines = file.readlines()
rankList={}
for line in lines:
    qid=int(line.split(" ")[0])
    if qid not in rankList:
        rankList[qid]=[]
    rankList[qid].append(line)
file.close()
file = open("intermidiateRankedLists.txt","r")
lines = file.readlines()
rankList2={}
for line in lines:
    qid=int(line.split(" ")[0])
    if qid not in rankList2:
        rankList2[qid]=[]
    rankList2[qid].append(line)
file.close()

for key in rankList2.keys():
    for i in range(0, len(rankList2[key])):
        rankList[key][i]=rankList2[key][i]
file=open("myNewRankedLists.txt","w")
for key in rankList.keys():
    for i in range(0, len(rankList[key])):
        file.write(rankList[key][i])
file.close()


In [3]:
score_data={}
label_data = {}
query={}
file1 = open("uploadedFiles/"+file_name+".txt","r")
for x in file1:
    curr_row=x.split(" ")
    qid=int(curr_row[1].split(":")[1])
    date=float(curr_row[14].split(":")[1])
    index2=int(curr_row[26].split("_")[0])-1
    if qid not in score_data:
        label_data[qid] = {}
        score_data[qid] = []
        query[qid]=curr_row[26].split("_")[2]
    label_data[qid][index2+1] = int(curr_row[0])
    score_data[qid].append([0,date,index2+1])
file1.close()

In [ ]:
f = open("myNewRankedLists.txt","r")
f1 = f.readlines()
ans = {}
for x in f1:
    y = x.split()
    value = y[len(y)-2]
    q_id = y[0]
    z = x.split('_')
    i = z[0]
    i = i.split()
    # print(i)
    i = i[4]
    for v in range(len(score_data[int(q_id)])):
        # print(score_data[int(q_id)][v][2])
        # print(i)
        if score_data[int(q_id)][v][2] == int(i):
            score_data[int(q_id)][v][0] = float(value)
            break
for id in score_data.keys():
    score_data[id].sort(key = lambda sub: (-sub[0], sub[1]))